Part 1 简单多线程程序的创建

Reference: _Programming with POSIX Threads_ by David R.Butenhof
# 建立和使用线程
线程的生命周期：
* 就绪（ready）：线程能够运行，但在等待可用的处理器。
* 运行（running）：线程正在运行
* 阻塞（blocked）：线程由于等待处理器外的其他条件无法运行，如条件变量的改变、加锁互斥量或I/O操作结束
* 终止（terminated）：线程从起始函数中返回，或调用`pthread_exit`,或被取消。不是被分离，也不是被连接，一旦线程被分离或者连接，它就可以被收回。

相关函数:

In [ ]:
#include <pthread.h>

pthread_t pthread;
int pthread_equal(pthread_t t1, pthread_t t2);
int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_func)(void *),void *arg);
pthread_t pthread_self(void);
int sched_yield(void);
int pthread_exit(void *val_ptr);
int pthread_detach(pthread_t thread);
int pthread_join(pthread_t thread, void **val_ptr);

C使用线程标识符ID来表示线程，线程ID属于封装的的`pthread_t`类型。
`pthreads`线程通过调用用户提供的某些函数开始。这个函数应该只有一个`void *`类型的函数,并且返回`void *`类型


# 同步——互斥量
如果两个线程同时访问共享数据，就可能会有问题，因为一个线程可能在另一个线程修改共享数据的过程中使用该数据，并认为共享数据保持不变。
看如下的例子：

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

#define ll long long
#define N 50000

ll sum=0;

void *sum_runner(void * arg){
    ll offset=*(ll *)arg;
    for(ll i=0;i<=N;i++)   sum+=offset;
    pthread_exit(NULL);
}

int main(int argc, char *argv[]){
    int offset1=1,offset2=-1;
    pthread_t pid1,pid2;
    
    pthread_create(&pid1,NULL,sum_runner,&offset1);
    pthread_create(&pid2,NULL,sum_runner,&offset2);
    
    pthread_join(pid1,NULL);
    pthread_join(pid2,NULL);
    
    //Expected output: "Sum is 0"
    printf("Sum is %lld\n",sum);
    return 0;
}

运行这个程序，可以发现，它是计算不出正确结果的。

检测这个程序，发现的确有data race的情况发生
```
$  gcc test.c -pthread -fsanitize=thread
$  ./a.out
==================
WARNING: ThreadSanitizer: data race (pid=22837)
  Write of size 8 at 0x000106cd6058 by thread T2:
    #0 sum_runner <null>:1600784 (a.out:x86_64+0x100000db1)

  Previous write of size 8 at 0x000106cd6058 by thread T1:
    #0 sum_runner <null>:1600784 (a.out:x86_64+0x100000db1)

  Location is global 'sum' at 0x000106cd6058 (a.out+0x000100002058)

  Thread T2 (tid=5183121, running) created by main thread at:
    #0 pthread_create <null>:1600864 (libclang_rt.tsan_osx_dynamic.dylib:x86_64h+0x2a9cd)
    #1 main <null>:1600864 (a.out:x86_64+0x100000e6d)

  Thread T1 (tid=5183119, finished) created by main thread at:
    #0 pthread_create <null>:1600864 (libclang_rt.tsan_osx_dynamic.dylib:x86_64h+0x2a9cd)
    #1 main <null>:1600864 (a.out:x86_64+0x100000e50)

SUMMARY: ThreadSanitizer: data race (a.out:x86_64+0x100000db1) in sum_runner
==================
Sum is -5952196224051314688
ThreadSanitizer: reported 1 warnings
Abort trap: 6
```

使线程同步的常用方法就是确保对相关数据的内存访问“互斥”地进行，即一次只能允许一个线程写数据，其他线程必须等待。
同步不仅在修改数据时重要，当线程需要读取其他线程写入的数据时，而且数据写入的顺序也有影响时，同样需要同步。

"信号灯”：互斥量 mutex(mutual exclusion)
## 创建和销毁
互斥量：`pthread_mutex_t`类型，不能拷贝互斥量变量，但可以拷贝其指针

### 静态初始化

In [ ]:
/* mutex statically initialized with default attributes */
typedef struct mystruct {
    pthread_mutex_t mutex;/* Protects access to value */
    int             value;/* Access protected by mutex */
} my_struct_t;

my_struct_t data = {PTHREAD_MUTEX_INITIALIZER, 0}; //No need to destory

### 动态初始化

In [ ]:
my_struct_t *data2=malloc(sizeof(data2));

pthread_mutex_init(&data->mutex,NULL);

//Do something here...

pthread_mutex_destroy(&data->mutex,NULL);

free(data);

**当确信没有线程在互斥量上阻塞时，可以立即释放它**

## 加锁和解锁
